# Manejo de Oclusiones

In [1]:
import cv2
from ultralytics import YOLO
from collections import deque
import numpy as np
import time
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
from enum import Enum

# Parámetros ajustables
MODEL_PATH = "/home/jhamilcr/Documents/proyecto-sis330/detection-model/train-files/best.pt"
MIN_CONF = 0.8
STABILITY_FRAMES = 20
EXCLUDED_CLASS = "hand"

# Parámetros para tracking de objetos
IOU_THRESHOLD = 0.35
CENTER_DISTANCE_THRESHOLD = 80
SIZE_RATIO_THRESHOLD = 0.5
OVERLAP_THRESHOLD = 0.5

# Parámetros específicos para carrito de compras
PRODUCT_TIMEOUT = 3.0
MIN_DETECTION_FRAMES = 8
OCCLUSION_TOLERANCE = 0.55
REMOVAL_CONFIRMATION_FRAMES = 30

# Parámetros de procesamiento optimizado
PROCESS_EVERY_N_FRAMES = 3
FRAME_SKIP_ON_STABLE = 5

# NUEVOS PARÁMETROS PARA SISTEMA DE CAPAS
OCCLUSION_TIMEOUT = 1000.0  # Tiempo máximo que un producto puede estar ocluido
LAYER_DEPTH_THRESHOLD = 0.7  # Umbral para considerar que un producto está "detrás"
RECOVERY_FRAMES = 5  # Frames necesarios para confirmar que un producto reapareció
MAX_LAYERS = 5  # Máximo número de capas a manejar

class ProductState(Enum):
    """Estados posibles de un producto"""
    DETECTING = "detecting"      # Siendo detectado pero no confirmado
    VISIBLE = "visible"          # Confirmado y visible
    OCCLUDED = "occluded"       # Confirmado pero ocluido por otro objeto
    RECOVERING = "recovering"    # Reapareciendo después de oclusión
    REMOVED = "removed"         # Removido del carrito

@dataclass
class Product:
    """Representa un producto en el carrito con soporte para capas"""
    id: str
    class_name: str
    bbox: Tuple[int, int, int, int]  # x1, y1, x2, y2
    confidence: float
    first_seen: float
    last_seen: float
    last_visible: float  # Última vez que fue realmente visible (no ocluido)
    detection_count: int
    confirmed: bool
    removal_count: int
    state: ProductState
    layer: int  # Capa estimada (0 = frente, mayor = más atrás)
    occlusion_start: Optional[float]  # Cuándo comenzó la oclusión
    recovery_count: int  # Contador para confirmar recuperación
    occluded_by: List[str]  # IDs de productos que lo están ocluyendo
    historical_positions: deque  # Historial de posiciones para predecir ubicación

class LayeredShoppingCart:
    """Carrito de compras con manejo de capas para oclusión"""
    
    def __init__(self):
        self.products: Dict[str, Product] = {}
        self.next_id = 1
        self.detection_history = deque(maxlen=STABILITY_FRAMES)
        self.last_stable_count = 0
        self.stability_counter = 0
        self.last_detection_time = 0
        self.spatial_grid = {}  # Grid para análisis espacial más eficiente
        
    def should_process_frame(self, frame_count: int, current_time: float) -> bool:
        """Determina si debe procesar este frame basado en optimizaciones"""
        if frame_count < PROCESS_EVERY_N_FRAMES * 2:
            return True
        
        if frame_count % PROCESS_EVERY_N_FRAMES != 0:
            return False
        
        current_count = len([p for p in self.products.values() 
                           if p.state in [ProductState.VISIBLE, ProductState.OCCLUDED]])
        
        if current_count == self.last_stable_count:
            self.stability_counter += 1
        else:
            self.stability_counter = 0
        
        self.last_stable_count = current_count
        
        if self.stability_counter > 10:
            return frame_count % (PROCESS_EVERY_N_FRAMES + FRAME_SKIP_ON_STABLE) == 0
        
        return True
    
    def get_detection_interval(self) -> int:
        """Retorna el intervalo actual de detección basado en estabilidad"""
        if self.stability_counter > 10:
            return PROCESS_EVERY_N_FRAMES + FRAME_SKIP_ON_STABLE
        return PROCESS_EVERY_N_FRAMES
    
    def generate_product_id(self, class_name: str) -> str:
        """Genera un ID único para un producto"""
        product_id = f"{class_name}_{self.next_id}"
        self.next_id += 1
        return product_id
    
    def compute_iou(self, boxA: Tuple, boxB: Tuple) -> float:
        """Calcula IoU entre dos bounding boxes"""
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        interW = max(0, xB - xA)
        interH = max(0, yB - yA)
        interArea = interW * interH

        if interArea == 0:
            return 0.0

        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
        unionArea = boxAArea + boxBArea - interArea

        return interArea / unionArea if unionArea > 0 else 0.0
    
    def compute_center_distance(self, boxA: Tuple, boxB: Tuple) -> float:
        """Calcula distancia entre centros de dos bounding boxes"""
        centerA = ((boxA[0] + boxA[2]) / 2, (boxA[1] + boxA[3]) / 2)
        centerB = ((boxB[0] + boxB[2]) / 2, (boxB[1] + boxB[3]) / 2)  # Fixed: boxB[0] to boxB[2]
        return np.sqrt((centerA[0] - centerB[0])**2 + (centerA[1] - centerB[1])**2)
    
    def estimate_depth_layer(self, product_id: str, detections: List[Tuple]) -> int:
        """Estima la capa de un producto basado en las relaciones de oclusión"""
        # Construir grafo de oclusión usando el campo occluded_by
        occlusion_graph = {pid: set() for pid in self.products}
        for pid, product in self.products.items():
            if product.state == ProductState.REMOVED:
                continue
            for occluder_id in product.occluded_by:
                # Extraer el ID del producto ocluyente (si es un producto registrado)
                for other_pid, other_product in self.products.items():
                    if other_product.state == ProductState.REMOVED:
                        continue
                    if occluder_id.startswith(other_product.class_name):
                        occlusion_graph[pid].add(other_pid)
                        break
        
        # Asignar capas basadas en el grafo
        return self.assign_layers_from_occlusion_graph(occlusion_graph, product_id)
    
    def assign_layers_from_occlusion_graph(self, occlusion_graph: Dict[str, set], product_id: str) -> int:
        """Asigna una capa a un producto basado en el grafo de oclusión"""
        if product_id not in occlusion_graph:
            return 0
        
        # Usar búsqueda en profundidad para calcular la profundidad máxima
        visited = set()
        memo = {}  # Cache para optimizar cálculos recursivos
        
        def get_max_depth(pid: str) -> int:
            if pid in memo:
                return memo[pid]
            if pid in visited:
                return 0  # Evitar ciclos
            visited.add(pid)
            max_depth = 0
            for occluder in occlusion_graph[pid]:
                max_depth = max(max_depth, get_max_depth(occluder) + 1)
            visited.remove(pid)
            memo[pid] = max_depth
            return max_depth
        
        layer = get_max_depth(product_id)
        return min(layer, MAX_LAYERS - 1)
    
    def compute_overlap_ratio(self, boxA: Tuple, boxB: Tuple) -> Tuple[float, float]:
        """Calcula qué porcentaje de cada box está solapado"""
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        interW = max(0, xB - xA)
        interH = max(0, yB - yA)
        interArea = interW * interH

        if interArea == 0:
            return 0.0, 0.0

        areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

        overlapA = interArea / areaA if areaA > 0 else 0
        overlapB = interArea / areaB if areaB > 0 else 0

        return overlapA, overlapB
    
    def predict_occluded_position(self, product: Product) -> Tuple[int, int, int, int]:
        """Predice la posición de un producto ocluido basado en su historial"""
        if len(product.historical_positions) < 2:
            return product.bbox
        
        # Calcular tendencia de movimiento
        recent_positions = list(product.historical_positions)[-3:]
        if len(recent_positions) < 2:
            return product.bbox
        
        # Calcular velocidad promedio
        dx_total = dy_total = 0
        for i in range(1, len(recent_positions)):
            prev_center = ((recent_positions[i-1][0] + recent_positions[i-1][2]) / 2,
                          (recent_positions[i-1][1] + recent_positions[i-1][3]) / 2)
            curr_center = ((recent_positions[i][0] + recent_positions[i][2]) / 2,
                          (recent_positions[i][1] + recent_positions[i][3]) / 2)
            dx_total += curr_center[0] - prev_center[0]
            dy_total += curr_center[1] - prev_center[1]
        
        # Velocidad promedio
        velocity_frames = len(recent_positions) - 1
        avg_dx = dx_total / velocity_frames if velocity_frames > 0 else 0
        avg_dy = dy_total / velocity_frames if velocity_frames > 0 else 0
        
        # Predecir nueva posición
        last_bbox = recent_positions[-1]
        width = last_bbox[2] - last_bbox[0]
        height = last_bbox[3] - last_bbox[1]
        
        predicted_center_x = (last_bbox[0] + last_bbox[2]) / 2 + avg_dx
        predicted_center_y = (last_bbox[1] + last_bbox[3]) / 2 + avg_dy
        
        return (
            int(predicted_center_x - width/2),
            int(predicted_center_y - height/2),
            int(predicted_center_x + width/2),
            int(predicted_center_y + height/2)
        )
    
    def analyze_occlusions(self, detections: List[Tuple], current_time: float):
        """Analiza qué productos están siendo ocluidos, respetando orden de entrada y capas"""
        # Inicializar occluded_by con los valores actuales
        temp_occluded_by = {pid: product.occluded_by.copy() for pid, product in self.products.items()}
        
        # Crear una copia de las detecciones disponibles
        available_detections = detections.copy()
        used_detections = set()
        
        # Asignar detecciones a productos visibles o recuperándose primero
        for product_id, product in self.products.items():
            if product.state in [ProductState.VISIBLE, ProductState.RECOVERING]:
                best_score = 0
                best_idx = -1
                for i, detection in enumerate(available_detections):
                    if i in used_detections:
                        continue
                    if self.is_same_product(product, detection):
                        class_name, x1, y1, x2, y2, conf = detection
                        det_bbox = (x1, y1, x2, y2)
                        iou = self.compute_iou(product.bbox, det_bbox)
                        center_dist = self.compute_center_distance(product.bbox, det_bbox)
                        score = iou * 0.6 + (1 - min(center_dist / CENTER_DISTANCE_THRESHOLD, 1.0)) * 0.4
                        if score > best_score:
                            best_score = score
                            best_idx = i
                if best_idx != -1:
                    used_detections.add(best_idx)
        
        # Ordenar productos por first_seen (ascendente) para procesar los más antiguos primero
        sorted_products = sorted(
            self.products.items(),
            key=lambda x: x[1].first_seen
        )
        
        # Analizar oclusiones respetando orden de entrada y capas
        for product_id, product in sorted_products:
            if product.state == ProductState.REMOVED:
                temp_occluded_by[product_id] = []
                continue
                
            # Usar posición predicha si está ocluido
            check_bbox = self.predict_occluded_position(product) if product.state == ProductState.OCCLUDED else product.bbox
            
            is_occluded = False
            occluding_objects = []
            
            # Verificar superposición con otros productos
            for other_pid, other_p in sorted_products:
                if other_pid == product_id or other_p.state == ProductState.REMOVED:
                    continue
                    
                # Solo considerar como occluder si está en una capa superior (menor valor de layer)
                # o si fue detectado después (mayor first_seen)
                if other_p.layer >= product.layer and other_p.first_seen <= product.first_seen:
                    continue
                    
                other_bbox = self.predict_occluded_position(other_p) if other_p.state == ProductState.OCCLUDED else other_p.bbox
                overlap_ratio = self.compute_overlap_ratio(check_bbox, other_bbox)[0]
                
                if overlap_ratio > OCCLUSION_TOLERANCE:
                    occluding_objects.append(other_pid)
                    is_occluded = True
                    print(f"[DEBUG] {product_id} potencialmente ocluido por {other_pid} | OverlapRatio: {overlap_ratio:.3f} | Layers: {product.layer} vs {other_p.layer}")
            
            # Verificar superposición con detecciones no asignadas
            for i, detection in enumerate(available_detections):
                if i in used_detections:
                    continue
                class_name, x1, y1, x2, y2, conf = detection
                det_bbox = (x1, y1, x2, y2)
                
                overlap_ratio = self.compute_overlap_ratio(check_bbox, det_bbox)[0]
                
                if overlap_ratio > OCCLUSION_TOLERANCE:
                    matched_product_id = None
                    for other_pid, other_p in self.products.items():
                        if other_pid == product_id or other_p.state == ProductState.REMOVED:
                            continue
                        if self.is_same_product(other_p, detection) and other_p.layer < product.layer:
                            matched_product_id = other_pid
                            if matched_product_id not in occluding_objects:
                                occluding_objects.append(matched_product_id)
                            break
                    
                    is_occluded = True
                    if not matched_product_id:
                        occluder_id = f"{class_name}_{x1}_{y1}"
                        if occluder_id not in occluding_objects:
                            occluding_objects.append(occluder_id)
            
            temp_occluded_by[product_id] = occluding_objects
            
            # Actualizar estado de oclusión
            if is_occluded and product.state == ProductState.VISIBLE:
                product.state = ProductState.OCCLUDED
                product.occlusion_start = current_time
                product.removal_count = 0
                print(f"[DEBUG] OCLUIDO: {product.id} | OverlapRatio: {overlap_ratio:.3f} | Occluders: {occluding_objects}")
                print(f"[~] OCLUIDO: {product.id} por {len(occluding_objects)} objeto(s): {occluding_objects}")
            elif product.state == ProductState.OCCLUDED:
                is_detected = any(self.is_same_product(product, detection) 
                                for i, detection in enumerate(available_detections) 
                                if i not in used_detections)
                if is_detected and not is_occluded:
                    product.state = ProductState.RECOVERING
                    product.recovery_count = 0
                    product.occlusion_start = None
                    product.removal_count = 0
                    print(f"[^] RECUPERANDO: {product.id}")
                elif is_occluded:
                    product.removal_count = 0
                    print(f"[DEBUG] Producto {product_id} sigue OCLUIDO por: {occluding_objects}")
                elif not is_occluded and not is_detected:
                    product.removal_count += max(1, PROCESS_EVERY_N_FRAMES // 3)
                    print(f"[DEBUG] Producto ocluido {product_id} no ocluido ni detectado, removal_count: {product.removal_count}")
        
        # Actualizar occluded_by y limpiar referencias a productos removidos
        for product_id, product in self.products.items():
            product.occluded_by = [oid for oid in temp_occluded_by[product_id] 
                                if oid in self.products and self.products[oid].state != ProductState.REMOVED]
            if product.occluded_by:
                print(f"[DEBUG] {product_id} occluded_by actualizado: {product.occluded_by}")
    
    def is_same_product(self, product: Product, detection: Tuple) -> bool:
        """Determina si una detección corresponde a un producto existente"""
        class_name, x1, y1, x2, y2, conf = detection
        
        if product.class_name != class_name:
            return False
        
        det_bbox = (x1, y1, x2, y2)
        
        # Usar posición predicha si está ocluido
        check_bbox = self.predict_occluded_position(product) if product.state == ProductState.OCCLUDED else product.bbox
        
        iou = self.compute_iou(check_bbox, det_bbox)
        center_dist = self.compute_center_distance(check_bbox, det_bbox)
        overlap_a, overlap_b = self.compute_overlap_ratio(check_bbox, det_bbox)
        
        # Calcular similitud en tamaño
        size_ratio = min(
            (check_bbox[2] - check_bbox[0]) / (det_bbox[2] - det_bbox[0]),
            (det_bbox[2] - det_bbox[0]) / (check_bbox[2] - check_bbox[0])
        )
        
        # Penalización por capa, menos estricta para productos en detección inicial
        estimated_layer = self.estimate_depth_layer(product.id, [(class_name, x1, y1, x2, y2, conf)])
        layer_diff = abs(product.layer - estimated_layer)
        layer_penalty = 1.0 - (0.1 * layer_diff) if product.state != ProductState.DETECTING else 1.0  # No penalizar en DETECTING
        
        # Verificar consistencia con el historial de movimiento (opcional para nuevos productos)
        movement_consistency = 1.0
        if len(product.historical_positions) >= 2 and product.state != ProductState.DETECTING:
            last_pos = product.historical_positions[-1]
            second_last_pos = product.historical_positions[-2]
            last_dx = (last_pos[0] + last_pos[2]) / 2 - (second_last_pos[0] + second_last_pos[2]) / 2
            last_dy = (last_pos[1] + last_pos[3]) / 2 - (second_last_pos[1] + second_last_pos[3]) / 2
            curr_dx = (det_bbox[0] + det_bbox[2]) / 2 - (last_pos[0] + last_pos[2]) / 2
            curr_dy = (det_bbox[1] + det_bbox[3]) / 2 - (last_pos[1] + last_pos[3]) / 2
            movement_diff = np.sqrt((last_dx - curr_dx)**2 + (last_dy - curr_dy)**2)
            movement_consistency = max(0, 1.0 - movement_diff / CENTER_DISTANCE_THRESHOLD)
        
        # Criterios combinados
        score = (
            iou * 0.5 +  # Aumentar peso de IoU para detecciones más confiables
            (1 - min(center_dist / CENTER_DISTANCE_THRESHOLD, 1.0)) * 0.3 +
            max(overlap_a, overlap_b) * 0.1 +
            size_ratio * 0.05 +  # Reducir peso de tamaño para no penalizar demasiado
            movement_consistency * 0.05
        ) * layer_penalty

        # After calculating score
        print(f"[DEBUG] is_same_product: {product.id} vs detection {class_name}_{x1}_{y1} | Score: {score:.3f} | IoU: {iou:.3f} | CenterDist: {center_dist:.1f} | SizeRatio: {size_ratio:.3f} | LayerPenalty: {layer_penalty:.3f} | MovementConsistency: {movement_consistency:.3f}")
        
        return score >= 0.6  # Reducir umbral para mejorar sensibilidad
    
    def find_matching_products(self, detections: List[Tuple]) -> Tuple[Dict[str, Optional[Tuple]], set]:
        """Encuentra qué detecciones corresponden a productos existentes"""
        matches = {}
        used_detections = set()
        
        # Ordenar productos por prioridad (visibles primero, luego por capa)
        sorted_products = sorted(
            self.products.items(),
            key=lambda x: (
                0 if x[1].state == ProductState.VISIBLE else
                1 if x[1].state == ProductState.RECOVERING else
                2 if x[1].state == ProductState.OCCLUDED else 3,
                x[1].layer
            )
        )
        
        for product_id, product in sorted_products:
            if product.state == ProductState.REMOVED:
                matches[product_id] = None
                continue
                
            best_match = None
            best_score = 0
            best_idx = -1
            
            for i, detection in enumerate(detections):
                if i in used_detections:
                    continue
                
                if self.is_same_product(product, detection):
                    class_name, x1, y1, x2, y2, conf = detection
                    det_bbox = (x1, y1, x2, y2)
                    
                    # Usar posición apropiada para el cálculo de score
                    check_bbox = self.predict_occluded_position(product) if product.state == ProductState.OCCLUDED else product.bbox
                    
                    iou = self.compute_iou(check_bbox, det_bbox)
                    center_dist = self.compute_center_distance(check_bbox, det_bbox)
                    
                    # Ajustar peso para productos ocluidos
                    if product.state == ProductState.OCCLUDED:
                        score = iou * 0.7 + (1 - min(center_dist / (CENTER_DISTANCE_THRESHOLD * 1.5), 1.0)) * 0.3
                    else:
                        score = iou * 0.6 + (1 - min(center_dist / CENTER_DISTANCE_THRESHOLD, 1.0)) * 0.4
                    
                    if score > best_score:
                        best_score = score
                        best_match = detection
                        best_idx = i
            
            if best_match is not None:
                matches[product_id] = best_match
                used_detections.add(best_idx)
                print(f"[DEBUG] Match encontrado para {product_id}: Score={best_score:.3f}")
            else:
                matches[product_id] = None
                print(f"[DEBUG] Sin match para {product_id}")
        
        return matches, used_detections
    
    def update_cart(self, detections: List[Tuple], current_time: float) -> Dict:
        """Actualiza el estado del carrito con nuevas detecciones"""
        changes = {
            'added': [],
            'updated': [],
            'removed': [],
            'maintained': [],
            'occluded': [],
            'recovered': []
        }

        # Analizar oclusiones primero
        self.analyze_occlusions(detections, current_time)
        
        # Actualizar capas para todos los productos confirmados
        for product_id in list(self.products.keys()):
            product = self.products[product_id]
            if product.confirmed and product.state != ProductState.REMOVED:
                product.layer = self.estimate_depth_layer(product_id, detections)
        
        # Encontrar matches entre productos existentes y detecciones
        matches, used_detections = self.find_matching_products(detections)
        
        # Actualizar productos existentes
        products_to_remove = []
        for product_id, detection in matches.items():
            product = self.products[product_id]
            
            if detection is not None:
                # Producto encontrado - actualizar
                class_name, x1, y1, x2, y2, conf = detection
                
                # Actualizar historial de posiciones
                new_bbox = (x1, y1, x2, y2)
                if len(product.historical_positions) == 0 or product.historical_positions[-1] != new_bbox:
                    product.historical_positions.append(new_bbox)
                
                product.bbox = new_bbox
                product.confidence = conf
                product.last_seen = current_time
                product.last_visible = current_time
                product.detection_count += 1
                product.removal_count = 0
                
                # Manejar transiciones de estado
                if product.state == ProductState.DETECTING:
                    if product.detection_count >= MIN_DETECTION_FRAMES:
                        product.confirmed = True
                        product.state = ProductState.VISIBLE
                        changes['added'].append(product_id)
                        print(f"[DEBUG] Confirmado como VISIBLE: {product_id}")
                        # Actualizar occluder IDs
                        for other_pid, other_p in self.products.items():
                            if other_pid != product_id and other_p.state != ProductState.REMOVED:
                                updated_occluders = []
                                for occluder_id in other_p.occluded_by:
                                    if occluder_id.startswith(product.class_name) and not occluder_id.startswith(product.id):
                                        updated_occluders.append(product_id)
                                        print(f"[DEBUG] Actualizado occluder en {other_pid}: {occluder_id} -> {product_id}")
                                    else:
                                        updated_occluders.append(occluder_id)
                                other_p.occluded_by = updated_occluders
                    else:
                        changes['updated'].append(product_id)
                
                elif product.state == ProductState.RECOVERING:
                    product.recovery_count += 1
                    if product.recovery_count >= RECOVERY_FRAMES:
                        product.state = ProductState.VISIBLE
                        changes['recovered'].append(product_id)
                    else:
                        changes['updated'].append(product_id)
                
                elif product.state == ProductState.VISIBLE:
                    changes['maintained'].append(product_id)
                    
                elif product.state == ProductState.OCCLUDED:
                    product.state = ProductState.RECOVERING
                    product.recovery_count = 1
                    product.removal_count = 0
                    changes['updated'].append(product_id)
                    print(f"[DEBUG] OCLUIDO detectado, pasando a RECOVERING: {product_id}")
            
            else:
                # Producto no encontrado
                if product.state == ProductState.OCCLUDED:
                    if product.occluded_by or (current_time - product.occlusion_start < 3.0):
                        changes['maintained'].append(product_id)
                        print(f"[DEBUG] Manteniendo OCLUIDO: {product_id} por {product.occluded_by or 'reciente oclusión'}")
                    else:
                        adjusted_removal_threshold = max(REMOVAL_CONFIRMATION_FRAMES // PROCESS_EVERY_N_FRAMES, 2)
                        if current_time - product.occlusion_start > OCCLUSION_TIMEOUT or product.removal_count >= adjusted_removal_threshold:
                            products_to_remove.append(product_id)
                            print(f"[DEBUG] Marcado para remover OCLUIDO: {product_id}, removal_count: {product.removal_count}")
                        else:
                            changes['maintained'].append(product_id)
                            print(f"[DEBUG] Manteniendo OCLUIDO: {product_id} (esperando recuperación)")
                
                elif product.state in [ProductState.VISIBLE, ProductState.RECOVERING]:
                    removal_increment = max(1, PROCESS_EVERY_N_FRAMES // 3)
                    product.removal_count += removal_increment
                    adjusted_removal_threshold = max(REMOVAL_CONFIRMATION_FRAMES // PROCESS_EVERY_N_FRAMES, 2)
                    
                    if product.confirmed and product.removal_count >= adjusted_removal_threshold:
                        products_to_remove.append(product_id)
                        print(f"[DEBUG] Marcado para remover VISIBLE/RECOVERING: {product_id}, removal_count: {product.removal_count}")
                    elif not product.confirmed and (current_time - product.last_seen) > PRODUCT_TIMEOUT/2:
                        products_to_remove.append(product_id)
                        print(f"[DEBUG] Marcado para remover DETECTING no confirmado: {product_id}")
                
                elif product.state == ProductState.DETECTING:
                    if (current_time - product.last_seen) > PRODUCT_TIMEOUT/2:
                        products_to_remove.append(product_id)
                        print(f"[DEBUG] Marcado para remover DETECTING: {product_id}")
        
        # Remover productos y limpiar referencias
        for product_id in products_to_remove:
            removed_product = self.products.pop(product_id)
            removed_product.state = ProductState.REMOVED
            if removed_product.confirmed:
                changes['removed'].append(product_id)
                print(f"[DEBUG] REMOVIDO: {product_id}")
            # Limpiar referencias en occluded_by
            for other_pid, other_p in self.products.items():
                if product_id in other_p.occluded_by:
                    other_p.occluded_by.remove(product_id)
                    print(f"[DEBUG] Eliminado {product_id} de occluded_by de {other_pid}")
        
        # Agregar nuevos productos
        for i, detection in enumerate(detections):
            if i not in used_detections:
                class_name, x1, y1, x2, y2, conf = detection
                product_id = self.generate_product_id(class_name)
                
                new_bbox = (x1, y1, x2, y2)
                historical_positions = deque(maxlen=10)
                historical_positions.append(new_bbox)
                
                new_product = Product(
                    id=product_id,
                    class_name=class_name,
                    bbox=new_bbox,
                    confidence=conf,
                    first_seen=current_time,
                    last_seen=current_time,
                    last_visible=current_time,
                    detection_count=1,
                    confirmed=False,
                    removal_count=0,
                    state=ProductState.DETECTING,
                    layer=self.estimate_depth_layer(product_id, detections),
                    occlusion_start=None,
                    recovery_count=0,
                    occluded_by=[],
                    historical_positions=historical_positions
                )
                
                self.products[product_id] = new_product
                changes['updated'].append(product_id)
                print(f"[DEBUG] Nuevo producto DETECTING: {product_id}")
        
        return changes
    
    def get_cart_summary(self) -> Dict:
        """Obtiene resumen del carrito actual con información de capas"""
        confirmed_products = {pid: p for pid, p in self.products.items() 
                            if p.confirmed and p.state != ProductState.REMOVED}
        visible_products = {pid: p for pid, p in confirmed_products.items() 
                          if p.state == ProductState.VISIBLE}
        occluded_products = {pid: p for pid, p in confirmed_products.items() 
                           if p.state == ProductState.OCCLUDED}
        recovering_products = {pid: p for pid, p in confirmed_products.items() 
                             if p.state == ProductState.RECOVERING}
        pending_products = {pid: p for pid, p in self.products.items() 
                          if p.state == ProductState.DETECTING}
        
        class_counts = {}
        for product in confirmed_products.values():
            class_counts[product.class_name] = class_counts.get(product.class_name, 0) + 1
        
        layer_distribution = {}
        for product in confirmed_products.values():
            layer_distribution[product.layer] = layer_distribution.get(product.layer, 0) + 1
        
        return {
            'confirmed_count': len(confirmed_products),
            'visible_count': len(visible_products),
            'occluded_count': len(occluded_products),
            'recovering_count': len(recovering_products),
            'pending_count': len(pending_products),
            'total_count': len(self.products),
            'class_counts': class_counts,
            'layer_distribution': layer_distribution,
            'confirmed_products': confirmed_products,
            'visible_products': visible_products,
            'occluded_products': occluded_products,
            'recovering_products': recovering_products,
            'pending_products': pending_products
        }

def get_detections(frame, model, min_conf):
    """Ejecuta YOLO y devuelve detecciones filtradas"""
    results = model.predict(source=frame, save=False, verbose=False)[0]
    detections = []
    
    for box in results.boxes:
        conf = float(box.conf)
        if conf < min_conf:
            continue
            
        cls_id = int(box.cls)
        cls_name = results.names[cls_id]
        
        if cls_name == EXCLUDED_CLASS:
            continue
            
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        detections.append((
            cls_name,
            int(x1), int(y1), int(x2), int(y2),
            conf
        ))
    
    return detections

def draw_layered_visualization(frame, cart: LayeredShoppingCart, processing_info: Dict):
    """Dibuja la visualización del carrito con información de capas"""
    summary = cart.get_cart_summary()
    
    # Colores por estado
    colors = {
        ProductState.VISIBLE: (0, 255, 0),      # Verde
        ProductState.OCCLUDED: (0, 165, 255),   # Naranja
        ProductState.RECOVERING: (255, 255, 0), # Amarillo
        ProductState.DETECTING: (0, 255, 255)   # Amarillo claro
    }
    
    # Dibujar productos por capa (desde atrás hacia adelante)
    max_layer = max([p.layer for p in summary['confirmed_products'].values()], default=0)
    
    for layer in range(max_layer, -1, -1):
        layer_products = [p for p in summary['confirmed_products'].values() if p.layer == layer]
        
        for product in layer_products:
            color = colors.get(product.state, (128, 128))
            x1, y1, x2, y2 = product.bbox
            # Grosor basado en capa (más grueso = más al frente)
            thickness = max(1, 3 - layer)
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            
            # Etiqueta con ID único y estado
            state_symbol = {
                ProductState.VISIBLE: "✓",
                ProductState.OCCLUDED: "◐",
                ProductState.RECOVERING: "↑",
                ProductState.DETECTING: "?"
            }.get(product.state, "")
            
            label = f"{product.id} L{product.layer} {state_symbol}"  # Mostrar ID único
            cv2.putText(frame, label, (x1, y1-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
            
            # Mostrar ocluyentes para productos ocluidos
            if product.state == ProductState.OCCLUDED and product.occluded_by:
                cv2.putText(frame, f"Ocl: {','.join(product.occluded_by)}", (x1, y1+20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
            
            # Mostrar predicción para productos ocluidos
            if product.state == ProductState.OCCLUDED:
                pred_bbox = cart.predict_occluded_position(product)
                px1, py1, px2, py2 = pred_bbox
                cv2.rectangle(frame, (px1, py1), (px2, py2), (128, 128, 128), 1, cv2.LINE_4)
    
    # Dibujar productos pendientes
    for product in summary['pending_products'].values():
        x1, y1, x2, y2 = product.bbox
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(frame, f"{product.class_name} ({product.detection_count}/{MIN_DETECTION_FRAMES})", 
                   (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
    
    # Panel de información
    y_offset = 30
    cv2.putText(frame, f"CARRITO: {summary['confirmed_count']} productos", 
               (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
    
    y_offset += 25
    cv2.putText(frame, f"Visibles: {summary['visible_count']} | Ocluidos: {summary['occluded_count']}", 
               (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    
    y_offset += 20
    if summary['recovering_count'] > 0:
        cv2.putText(frame, f"Recuperando: {summary['recovering_count']}", 
                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        y_offset += 20
    
    # Conteo por clase
    for class_name, count in summary['class_counts'].items():
        cv2.putText(frame, f"  {class_name}: {count}", 
                   (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        y_offset += 20
    
    # Distribución por capas
    if summary['layer_distribution']:
        y_offset += 10
        cv2.putText(frame, "Capas:", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
        y_offset += 15
        for layer, count in sorted(summary['layer_distribution'].items()):
            cv2.putText(frame, f"  L{layer}: {count}", 
                       (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1)
            y_offset += 15
    
    # Información de procesamiento
    frame_h, frame_w = frame.shape[:2]
    if processing_info['processed']:
        status_text = f"PROCESANDO (cada {processing_info['interval']} frames)"
        color = (0, 255, 0)
    else:
        status_text = f"SALTANDO ({processing_info['skipped']} saltados)"
        color = (100, 100, 100)
    
    cv2.putText(frame, status_text, (frame_w - 400, 25), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    # FPS y estabilidad
    cv2.putText(frame, f"FPS: {processing_info.get('fps', 0):.1f}", 
               (frame_w - 100, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    if cart.stability_counter > 5:
        cv2.putText(frame, "ESTABLE", (frame_w - 100, 65), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
    
    # Leyenda de estados
    legend_y = frame_h - 120
    cv2.putText(frame, "Estados:", (frame_w - 150, legend_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
    legend_y += 15
    cv2.putText(frame, "✓ Visible", (frame_w - 150, legend_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
    legend_y += 15
    cv2.putText(frame, "◐ Ocluido", (frame_w - 150, legend_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 165, 255), 1)
    legend_y += 15
    cv2.putText(frame, "↑ Recuperando", (frame_w - 150, legend_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1)
    legend_y += 15
    cv2.putText(frame, "? Detectando", (frame_w - 150, legend_y), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1)

def main():
    model = YOLO(MODEL_PATH)
    cart = LayeredShoppingCart()
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("No se pudo acceder a la cámara.")
        return

    print("=== SISTEMA DE CARRITO CON CAPAS PARA OCLUSIÓN ===")
    print("Controles:")
    print("  'q' - Salir")
    print("  'r' - Reiniciar carrito")
    print("  'i' - Mostrar información detallada")
    print("  'c' - Limpiar consola")
    print("  's' - Guardar estado del carrito")
    print("  'f' - Toggle procesamiento cada frame vs optimizado")
    print("  'l' - Mostrar información de capas")
    print("  'o' - Mostrar productos ocluidos")
    print(f"\nConfiguración actual:")
    print(f"  - Procesamiento cada {PROCESS_EVERY_N_FRAMES} frames")
    print(f"  - Timeout de oclusión: {OCCLUSION_TIMEOUT}s")
    print(f"  - Máximo {MAX_LAYERS} capas")
    print(f"  - Frames de recuperación: {RECOVERY_FRAMES}")
    
    show_info = False
    show_layers = False
    show_occluded = False
    frame_count = 0
    force_every_frame = False
    last_fps_time = time.time()
    fps_counter = 0
    current_fps = 0
    skipped_frames = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            current_time = time.time()
            frame_count += 1
            fps_counter += 1
            
            # Calcular FPS
            if current_time - last_fps_time >= 1.0:
                current_fps = fps_counter / (current_time - last_fps_time)
                fps_counter = 0
                last_fps_time = current_time
            
            # Determinar si procesar este frame
            should_process = force_every_frame or cart.should_process_frame(frame_count, current_time)
            
            processing_info = {
                'processed': should_process,
                'interval': cart.get_detection_interval(),
                'skipped': skipped_frames,
                'fps': current_fps
            }
            
            changes = {'added': [], 'updated': [], 'removed': [], 'maintained': [], 
                      'occluded': [], 'recovered': []}
            
            if should_process:
                detections = get_detections(frame, model, MIN_CONF)
                changes = cart.update_cart(detections, current_time)
                skipped_frames = 0
            else:
                skipped_frames += 1
                # Verificar timeouts para productos ocluidos
            
            # Dibujar visualización
            draw_layered_visualization(frame, cart, processing_info)
            
            # Mostrar cambios importantes
            if changes['added']:
                for product_id in changes['added']:
                    product = cart.products[product_id]
                    print(f"[+] AGREGADO: {product.class_name} (ID: {product_id}, Capa: {product.layer})")
            
            if changes['removed']:
                for product_id in changes['removed']:
                    print(f"[-] REMOVIDO: {product_id}")
            
            if changes['recovered']:
                for product_id in changes['recovered']:
                    product = cart.products[product_id]
                    print(f"[^] RECUPERADO: {product.class_name} (ID: {product_id})")
            
            # Información detallada (opcional)
            if show_info and frame_count % 30 == 0:
                summary = cart.get_cart_summary()
                print(f"\n--- Estado del Carrito (Frame {frame_count}) ---")
                print(f"Productos confirmados: {summary['confirmed_count']}")
                print(f"  - Visibles: {summary['visible_count']}")
                print(f"  - Ocluidos: {summary['occluded_count']}")
                print(f"  - Recuperando: {summary['recovering_count']}")
                print(f"Productos pendientes: {summary['pending_count']}")
                print(f"Estabilidad: {cart.stability_counter}")
                print(f"Intervalo actual: cada {cart.get_detection_interval()} frames")
                for class_name, count in summary['class_counts'].items():
                    print(f"  {class_name}: {count}")
                print("-" * 45)
            
            # Información de capas
            if show_layers and frame_count % 60 == 0:
                summary = cart.get_cart_summary()
                print(f"\n--- Distribución por capas ---")
                for layer in sorted(summary['layer_distribution'].keys()):
                    layer_products = [p for p in summary['confirmed_products'].values() if p.layer == layer]
                    products_info = []
                    for p in layer_products:
                        occluders = [pid for pid, other in summary['confirmed_products'].items()
                                     if p.id in other.occluded_by]
                        occluder_info = f"occluded by: {occluder_info}" if occluders else "top"
                        products_info.append(f"{p.id} ({p.class_name}, {p.state.value}, {occluder_info})")
                    print(f"Capa {layer}: {len(layer_products)} productos - {', '.join(products_info)}")
                print("-" * 30)
            
            # In main(), replace the show_occluded block with:
            if show_occluded and frame_count % 10 == 0:  # Cada 10 frames para logs frecuentes
                occluded = [p for p in cart.products.values() if p.state == ProductState.OCCLUDED]
                if occluded:
                    print(f"\n--- Productos Ocluidos ({len(occluded)}) ---")
                    for product in occluded:
                        occlusion_time = current_time - product.occlusion_start
                        first_seen_time = current_time - product.first_seen
                        print(f"{product.id}: {occlusion_time:.1f}s ocluido, Capa {product.layer}, removal_count: {product.removal_count}")
                        print(f"  Detectado hace: {first_seen_time:.1f}s")
                        if product.occluded_by:
                            occluders_info = []
                            for oid in product.occluded_by:
                                if oid in cart.products:
                                    occluder = cart.products[oid]
                                    occluders_info.append(f"{oid} (Capa {occluder.layer}, Detectado hace {current_time - occluder.first_seen:.1f}s)")
                                else:
                                    occluders_info.append(f"{oid} (Temporal)")
                            print(f"  Ocluido por: {', '.join(occluders_info)}")
                        else:
                            print(f"  Sin ocluyentes activos")
                        print(f"  Posición predicha: {cart.predict_occluded_position(product)}")
                    print("-" * 25)
            
            # Controles de teclado
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):
                cart = LayeredShoppingCart()
                print("\n[!] Carrito reiniciado")
            elif key == ord('i'):
                show_info = not show_info
                print(f"\nInformación detallada: {'ON' if show_info else 'OFF'}")
            elif key == ord('l'):
                show_layers = not show_layers
                print(f"\nInformación de capas: {'ON' if show_layers else 'OFF'}")
            elif key == ord('o'):
                show_occluded = not show_occluded
                print(f"\nInformación de oclusión: {'ON' if show_occluded else 'OFF'}")
            elif key == ord('c'):
                print("\n" * 50)
            elif key == ord('f'):
                force_every_frame = not force_every_frame
                mode = "CADA FRAME" if force_every_frame else "OPTIMIZADO"
                print(f"\nModo de procesamiento: {mode}")
            elif key == ord('s'):
                summary = cart.get_cart_summary()
                print(f"\n=== ESTADO ACTUAL DEL CARRITO ===")
                print(f"Total de productos: {summary['confirmed_count']}")
                print(f"  - Visibles: {summary['visible_count']}")
                print(f"  - Ocluidos: {summary['occluded_count']}")
                print(f"  - Recuperando: {summary['recovering_count']}")
                print("Productos por categoría:")
                for class_name, count in summary['class_counts'].items():
                    print(f"  - {class_name}: {count} unidad(es)")
                print("Distribución por capas:")
                for layer, count in sorted(summary['layer_distribution'].items()):
                    print(f"  - Capa {layer}: {count} producto(s)")
                print(f"Procesamiento: cada {cart.get_detection_interval()} frames")
                print(f"FPS actual: {current_fps:.1f}")
                print("=" * 40)

            cv2.imshow("Carrito de Compras - Sistema de Capas", frame)

    except KeyboardInterrupt:
        print("\nTerminando por interrupción del usuario...")

    # Mostrar resumen final
    final_summary = cart.get_cart_summary()
    print(f"\n=== RESUMEN FINAL ===")
    print(f"Productos en el carrito: {final_summary['confirmed_count']}")
    print(f"  - Visibles: {final_summary['visible_count']}")
    print(f"  - Ocluidos: {final_summary['occluded_count']}")
    for class_name, count in final_summary['class_counts'].items():
        print(f"  - {class_name}: {count}")
    print(f"Distribución final por capas:")
    for layer, count in sorted(final_summary['layer_distribution'].items()):
        print(f"  - Capa {layer}: {count} producto(s)")
    print(f"FPS promedio: {current_fps:.1f}")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

=== SISTEMA DE CARRITO CON CAPAS PARA OCLUSIÓN ===
Controles:
  'q' - Salir
  'r' - Reiniciar carrito
  'i' - Mostrar información detallada
  'c' - Limpiar consola
  's' - Guardar estado del carrito
  'f' - Toggle procesamiento cada frame vs optimizado
  'l' - Mostrar información de capas
  'o' - Mostrar productos ocluidos

Configuración actual:
  - Procesamiento cada 3 frames
  - Timeout de oclusión: 1000.0s
  - Máximo 5 capas
  - Frames de recuperación: 5


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/jhamilcr/Documents/proyecto-sis330/env/lib/python3.12/site-packages/cv2/qt/plugins"


[DEBUG] Nuevo producto DETECTING: flan-vainilla-kris-120g_1
[DEBUG] is_same_product: flan-vainilla-kris-120g_1 vs detection flan-vainilla-kris-120g_666_240 | Score: 0.997 | IoU: 0.997 | CenterDist: 0.5 | SizeRatio: 1.000 | LayerPenalty: 1.000 | MovementConsistency: 1.000
[DEBUG] Match encontrado para flan-vainilla-kris-120g_1: Score=0.996
[DEBUG] is_same_product: flan-vainilla-kris-120g_1 vs detection flan-vainilla-kris-120g_666_240 | Score: 1.000 | IoU: 1.000 | CenterDist: 0.0 | SizeRatio: 1.000 | LayerPenalty: 1.000 | MovementConsistency: 1.000
[DEBUG] Match encontrado para flan-vainilla-kris-120g_1: Score=1.000
[DEBUG] is_same_product: flan-vainilla-kris-120g_1 vs detection flan-vainilla-kris-120g_666_239 | Score: 0.997 | IoU: 0.997 | CenterDist: 0.5 | SizeRatio: 1.000 | LayerPenalty: 1.000 | MovementConsistency: 1.000
[DEBUG] Match encontrado para flan-vainilla-kris-120g_1: Score=0.996
[DEBUG] is_same_product: flan-vainilla-kris-120g_1 vs detection flan-vainilla-kris-120g_666_239 |